In [22]:
import music21 as m21
import re
from collections import Counter
from swalign_local import *

In [23]:
# import the score
score = '~/facets-search-engine/data/Beethoven9thOdeToJoy.xml'
#score = '~/jupyternotebooks/bwv1046a.mei'
full_score = m21.converter.parse(score)
len(full_score.parts)

2

In [24]:
# get first voice as the first doc in alignment
m21_score = full_score.parts[0]

In [25]:
# get the second voice as the second doc in alignment
compared_score = full_score.parts[1]

In [26]:
if len(full_score.parts) > 2:
    print("More than 2 voices! We should compare all the voices.")

In [27]:
def get_intervals(m21_score):
        
        noteoffset = []
        beatstrength = []
        
        dict_wordtonum = {"Unison": '2', "Second": '2', "Third": '3', "Fourth": '4', "Fifth": '5', "Sixth": '6', "Seventh": '7'}
        #P.S.: Unison as "2" instead of "0" for a reason, see explanation later
        
        # encoding the diatonic intervals. not 1A and 1D are represented as 2A and 2D here...
        dict_encode_dia_intervals = {"7D": 'A', '6D': 'B', '5D': 'C', '4D':'D', '3D':'E', '2D':'F', 
                                     '2A':'G', '3A':'H', '4A':'I', '5A':'J', '6A': 'K', '7A':'L', '0A': 'M'}
        
        diatonic_intervals = []
        
        previous_note = None
        
        # Scan the items        
        for thisnote in m21_score.recurse().notes:
            
            # We ignore rests
            if thisnote.isRest: 
                # If the rest is a full measure, part of a multi-measure rest: we need to adjust
                continue
            
            noteoffset.append(thisnote.offset)
            beatstrength.append(thisnote.beatStrength)
            #"duration:", thisnote.duration.quarterLength)
            #print("offset:", thisnote.offset, "beat strength", thisnote.beatStrength)
            if previous_note is None:
                previous_note = thisnote
            else:
                if thisnote.isNote and previous_note.isNote:
                    # gap = number of semi-tones of the current interval 
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isNote:
                    gap = thisnote.root().diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isChord:
                    gap = thisnote.root().diatonicNoteNum - previous_note.root().diatonicNoteNum
                elif thisnote.isNote and previous_note.isChord:
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.root().diatonicNoteNum
                # if a pitch change is detected
                    
                if gap != 0:
                    if gap > 0:
                        #  if the semi-tone difference between the current and the previous item > 0, it is an ascending interval.
                        direction = 'A'
                    else:
                        #  otherwise, it is a descending interval.
                        direction = 'D'

                    # Get intervals using music21
                    """
                            "directedSimpleNiceName" examples: "Descending Doubly-Diminished Fifth", "Ascending Perfect Fourth", "Ascending Doubly-Augmented Fourth"
                            "simpleName" examples: dd5, P5, AA4. There's no direction information
                            Since it only executes when a pitch interval is detected, "unison" refers to an augmented unison, a.k.a minor second
                    """
                    # take intervals between root notes if there exists any chord
                    if previous_note.isChord:
                        startnote = previous_note.root()
                    else:
                        startnote = previous_note
                    if thisnote.isChord:
                        endnote = thisnote.root()
                    else:
                        endnote = thisnote
                        
                    m21_interval_directed = m21.interval.Interval(noteStart=startnote, noteEnd=endnote).directedSimpleNiceName

                    arr_diatonic = m21_interval_directed.split(" ")

                    m21_generic = dict_wordtonum[arr_diatonic[-1]]
                    
                    # m21_interval: 2A, 3D etc...
                    m21_interval = m21_generic+direction
                    # to make each m21_interval unique, show direction and each as single character in string, we encode the diatonic intervals as letters
                    encode_interval = dict_encode_dia_intervals[m21_interval]
                    diatonic_intervals.append(encode_interval)

                else:
                    # We take the interval between two consecutive pitches as 0A
                    encode_interval = dict_encode_dia_intervals['0A']
                    diatonic_intervals.append(encode_interval)
                previous_note = thisnote
                    

        return diatonic_intervals, noteoffset, beatstrength


In [28]:
def to_string(list_intervals):
    
    string = ""
    for i in list_intervals:
        string += str(i)
    print(len(string))
    return string

In [29]:
def swm_alignment(string1, string2):
    
    match = 2
    mismatch = -2
    scoring = NucleotideScoringMatrix(match, mismatch)
    sw = LocalAlignment(scoring)#, gap_extension_penalty = -5) 
    
    alignment = sw.align(string2, string1)
    alignment_strings = alignment.dump()
    return alignment_strings


In [30]:
def find_pattern_beat_weighed(beatstrength, string, pattern):
    
    weighedscore = 0
    count = 0
    #match = re.search(pattern, string)
    #if match != None:
    #    print("no match in this one")
    for m in re.finditer(pattern, string):
        count +=1
        if beatstrength[m.start()] == 1.0:
            weighedscore += 1.5
        elif beatstrength[m.start()] >= 0.5:
            weighedscore += 1
        else:
            weighedscore += beatstrength[m.start()]
        #print(m.start(), beatstrength[m.start()])
    return weighedscore, count

def count_pattern_beat_weighed(reducedfinalpat, string1, string2, beatstrength1, beatstrength2):
    
    weighed_score = {}
    count_pattern = {}
    
    for pattern in reducedfinalpat: # finalpattern
        # initialize the weighed value
        weighed_score[pattern] = 0
        count_pattern[pattern] = 0

        # find the pattern in the first score
        weighedscore, count = find_pattern_beat_weighed(beatstrength1, string1, pattern)
        weighed_score[pattern] += weighedscore
        count_pattern[pattern] += count
    
        # find the pattern in the second score
        weighedscore, count = find_pattern_beat_weighed(beatstrength2, string2, pattern)
        weighed_score[pattern] += weighedscore
        count_pattern[pattern] += count
    
    return count_pattern, weighed_score


In [31]:
def reduce_pattern_length(seq):
    
        # the function returns the longest substring that appeared at least once
        # if there is length 4 substring repeated 4 times and a length 7 substring repeated 2 times, we take the length 7
        best_performance = ""
        candidates = []
        for length in range(int(0.2*len(seq)), int(len(seq)*0.5)+1): 
            for start in range(0, len(seq)-length):
                # get all the substrings of this length within the string, save in candidates
                candidates.append(seq[start:start+length])
        count_can = {}
        for candidate in candidates:
                count_can[candidate] = seq.count(candidate)
                if count_can[candidate] > 1:
                    # if it is repeated more than once in the string
                    if len(candidate) > len(best_performance):
                        best_performance = candidate
                    elif len(candidate) == len(best_performance) and count_can[candidate] > count_can[best_performance]:
                        best_performance = candidate
        
        return best_performance

def check_reduction(item):
    
    frequent_substr = reduce_pattern_length(item)
    times = item.count(frequent_substr) 
    if len(frequent_substr) >= 4:
        # if it is a large part of the string, or it appeared more than 3 times in the string
        if len(frequent_substr) > 0.2*len(item) or times >= 3:
            print("extracted", frequent_substr, "from", item)
            # sucessfully reduced the pattern
            return frequent_substr
    return None


In [32]:
def reduce_length_of_the_pattern(item):
    
    # reduce the length of this item
    frequent_substr = check_reduction(item)
    #startpos = []
    
    if frequent_substr != None:
        if len(frequent_substr) > 23:
            # second attempt, if a frequent substring is still quite long
            new_frequent_substr = check_reduction(frequent_substr)
            if new_frequent_substr != None:
                # if the second attempt is a success, take the twice reduced pattern
                """
                # FIND START POS OF THE NEW ITEMS
                for m in re.finditer(new_frequent_substr, item):
                    print(frequent_substr, item, m.start(), pos)
                    startpos.append(pos+m.start())
                """
                return new_frequent_substr #, startpos
            else:
                # if the second attempt failed, take the result of first reduction
                """
                for m in re.finditer(frequent_substr, item):
                    print(frequent_substr, item, m.start(), pos)
                    startpos.append(pos+m.start())
                """
                return frequent_substr #, startpos
        else:
            # if the extracted pattern does not need further reduction
            """
            for m in re.finditer(frequent_substr, item):
                print(frequent_substr, item, m.start(), pos)
                startpos.append(pos+m.start()) 
            """
            return frequent_substr #, startpos
    
    # if the reduction failed, return the original, startpos will be an empty list
    return None #, startpos
    

In [33]:
"""
def add_item_to_list(startposinseq1, startpos, pattern_to_add, pattern1):
    
    # add the pattern to the list
    pattern1.append(pattern_to_add)
    
    if pattern_to_add not in startposinseq1:
        # save this pattern
        startposinseq1[pattern_to_add] = startpos
    else:
        # if this pattern already existed, add the new positions to the list
        startposinseq1[pattern_to_add] += startpos

    return startposinseq1, pattern1
"""

'\ndef add_item_to_list(startposinseq1, startpos, pattern_to_add, pattern1):\n    \n    # add the pattern to the list\n    pattern1.append(pattern_to_add)\n    \n    if pattern_to_add not in startposinseq1:\n        # save this pattern\n        startposinseq1[pattern_to_add] = startpos\n    else:\n        # if this pattern already existed, add the new positions to the list\n        startposinseq1[pattern_to_add] += startpos\n\n    return startposinseq1, pattern1\n'

In [34]:
def get_patterns_from_alignment(alignment_strings):
    
    fullseq1 = alignment_strings[0]
    pat1 = fullseq1.split('-')
    fullseq2 = alignment_strings[1]
    pat2 = fullseq2.split('-')

    pattern1 = []
    pattern2 = []
    
    # startpos_pat saves all the start position of the pattern(if it appears only once, then it's a list of one element)
    # this dictionary saves the pattern appears from the Xth note, instead of offset. 
    
    #startposinseq1_pat = {}
    #startposinseq2_pat = {}

    for pattern in pat1:
        if len(pattern) > 2:
            # Get rid of sequences shorter than 3 intervals
            if len(pattern) > 11:
                #if the pattern is too long, find longest repeated pattern within it
                newpattern = reduce_length_of_the_pattern(pattern)
                if newpattern != None:
                    # if the extraction succeeded
                    pattern1.append(newpattern)
                    #pattern1 = add_item_to_list(startposinseq1_pat, startpos, newpattern, pattern1)
                else:
                    # add the original position to the list
                    #startpos.append(pos)
                    print("not extractable:", pattern)
                    pattern1.append(pattern)
                    #pattern1 = add_item_to_list(startposinseq1_pat, startpos, pattern, pattern1)
            else:
                # no need to extract a shorter one from this pattern
                # add the original position to the list
                pattern1.append(pattern)
                #startposinseq1_pat, pattern1 = add_item_to_list(startposinseq1_pat, startpos, pattern, pattern1)

    for pattern in pat2:
        if len(pattern) > 2:
            if len(pattern) > 11:
                #if the pattern is too long, find longest repeated pattern within it
                newpattern = reduce_length_of_the_pattern(pattern)
                if newpattern != None:
                    # if the extraction succeeded
                    pattern2.append(newpattern)
                else:
                    print("not extractable:", pattern)
                    pattern2.append(pattern)
            else:
                # no need to extract a shorter one from this pattern
                pattern2.append(pattern)
    
    tempallpatterns = pattern1 + pattern2
    
    allthepattern_count = dict(Counter(tempallpatterns))

    # Get the number of distinctive ones
    allthepatterns = list(set(tempallpatterns))
    len(allthepatterns)
    
    return allthepatterns, allthepattern_count


In [35]:
def count_substringandparents(allthepatterns, allthepattern_count):
    
    issubstring = {}
    hassubstring = {}
    count_as_substring = {}
    count_as_parent = {}

    for i in range(0, len(allthepatterns)):
        for j in range(i+1, len(allthepatterns)):
            if allthepatterns[i] in allthepatterns[j]:
                # if pattern i is substring of pattern j
                parent = allthepatterns[j]
                kid = allthepatterns[i]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] = 1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1#times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 

            elif allthepatterns[j] in allthepatterns[i]:
                # if pattern j is substring of pattern i

                parent = allthepatterns[i]
                kid = allthepatterns[j]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] =1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1 #times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 #times

    return issubstring, hassubstring, count_as_substring, count_as_parent

In [36]:
# This counts the times each pattern appeared after original segmentation plus the times they are in other patterns,
# plus the times other patterns showed up in them.

def combined_count(sorthas, sortis, count_as_parent, count_as_substring, allthepattern_count):
    
    refined_combined = {}

    for item in sorthas:
        if item in sortis:
            # the ones that has substring and also are the substrings
            refined_combined[item] = count_as_parent[item] + count_as_substring[item] + allthepattern_count[item]
        else:
            # the ones that has substrings
            refined_combined[item] = count_as_parent[item] + allthepattern_count[item]

    # the patterns that are substrings of other patterns but none of the others are substrings of it
    for item in sortis:
        if item not in refined_combined:
            refined_combined[item] = count_as_substring[item] + allthepattern_count[item]
        # otherwise is already counted
    return refined_combined


In [37]:
def show_combined_result(finalpattern_combined, allthepattern_count):
    print(dict(Counter(finalpattern_combined)))
    for i in finalpattern_combined:
        print(i, allthepattern_count[i])

In [38]:
def filter_combined_patterns(combined):
    
    finalpattern = []
    for i in combined: 

        if len(i) < 3:
            # if the length is shorter than 4 notes, discard
            continue

        if combined[i] > 1:
            # keep the ones that has substring or is substring or appeared more than once
            finalpattern.append(i)
            
    finalpattern_combined = {}
        
    for thisone in finalpattern:
        finalpattern_combined[thisone] = combined[thisone]

    dict(Counter(finalpattern_combined))
        
    if len(finalpattern) > 20:
        # if more than 20 patterns, only take the top 20
        finalpattern_combined = dict(sorted(finalpattern_combined.items(), key = lambda x:-x[1], reverse = True)[-20:])
    
    return finalpattern_combined


In [39]:
def decode_results(finalpat, weighed_score, count_pattern, combined):
    
    decoded_patterns = []
    decoded_patterns_weighed = {}
    decoded_patterns_count = {}
    decoded_patterns_combined = {}
    
    decode_dia_intervals = {'A': '7D', 'B': '6D', 'C': '5D', 'D': '4D', 'E': '3D', 'F': '2D', 
                            'G': '2A', 'H': '3A', 'I': '4A', 'J': '5A', 'K': '6A', 'L': '7A', 'M': '0A'}

    # decode all the candidates for patterns
    for pattern in finalpat:
        trans_pattern = ""
        for letter in pattern:
            trans = decode_dia_intervals[letter]
            trans_pattern += trans

        decoded_patterns_weighed[trans_pattern] = weighed_score[pattern]
        decoded_patterns_count[trans_pattern] = count_pattern[pattern]
        decoded_patterns_combined[trans_pattern] = combined[pattern]
        decoded_patterns.append(trans_pattern)
        
    return decoded_patterns, decoded_patterns_combined, decoded_patterns_weighed, decoded_patterns_count


In [40]:
def decode_strings(string1, string2):
    
    decode_dia_intervals = {'A': '7D', 'B': '6D', 'C': '5D', 'D': '4D', 'E': '3D', 'F': '2D', 
                            'G': '2A', 'H': '3A', 'I': '4A', 'J': '5A', 'K': '6A', 'L': '7A', 'M': '0A'}
    
    decode_string1 = ""
    decode_string2 = ""

    for letter in string1:
        trans = decode_dia_intervals[letter]
        decode_string1+=trans

    for letter in string2:
        trans = decode_dia_intervals[letter]
        decode_string2+=trans
    
    return decode_string1, decode_string2

In [41]:
def process_two_voices(m21_score, compared_score):
    
    # Get diatonic intervals, beat strength and other information from scores.
    dia_intervals1, noteoffset1, beatstrength1 = get_intervals(m21_score)
    dia_intervals2, noteoffset2, beatstrength2 = get_intervals(compared_score)

    string1 = to_string(dia_intervals1)
    string2 = to_string(dia_intervals2)

    # Smith waterman alignment of two strings
    alignment_strings = swm_alignment(string1, string2)
    
    """
    startposinseq1 saves the starting position of each pattern in seq1
    startposinseq2 saves the starting position of each pattern in seq2
    allthepatterns is a list of unique candidate patterns(that will go through parent-child count)
    allthepatterns_count counts the times each candidate pattern in allthepatterns appear in both seqs.       
    """
    
    # get patterns and their start positions, and reduce the long ones to "longest repeated substring"
    allthepatterns, allthepattern_count = get_patterns_from_alignment(alignment_strings)
        
    # find substring relationships between all pairs of candidate patterns
    issubstring, hassubstring, count_as_substring, count_as_parent = count_substringandparents(allthepatterns, allthepattern_count)
    
    # sort the ones that are substrings
    sortis = dict(Counter(issubstring))
    # sort the ones that have substrings
    sorthas = dict(Counter(hassubstring))
    
    combined = combined_count(sorthas, sortis, count_as_parent, count_as_substring, allthepattern_count)
    
    # filter out the short ones
    top20patterns_combined = filter_combined_patterns(combined)
    
    # just print what's going on
    show_combined_result(top20patterns_combined, allthepattern_count)
    
    # count patterns in origianl strings and weigh them with their beat strength
    count_pattern, weighed_scores = count_pattern_beat_weighed(top20patterns_combined, string1, string2, beatstrength1, beatstrength2)
    
    # Decode patterns
    decoded_patterns, decoded_patterns_combined, decoded_patterns_weighed, decoded_patterns_count = decode_results(top20patterns_combined, weighed_scores, count_pattern, combined)
    
    decode_string1, decode_string2 = decode_strings(string1, string2)
        
    for pattern in decoded_patterns:
        print("Pattern:", pattern)

        print("importance score:", decoded_patterns_combined[pattern],
              "weighed score:", decoded_patterns_weighed[pattern], 
              "occurrence:", decoded_patterns_count[pattern],
              "average weigh:", decoded_patterns_weighed[pattern]/decoded_patterns_count[pattern])

        match1 = re.search(pattern, decode_string1)
        if match1 != None:
            print("first occurrence in decoded string1: % s, % s" % (match1.start(), match1.end()))
        else:
            print("the pattern does not exist in decoded string1.")
        match2 = re.search(pattern, decode_string2)
        if match2 != None:
            print("first occurrence in decoded string2: % s, % s" % (match2.start(), match2.end()))
        else:
            print("the pattern does not exist in decoded string2.")
            
    return decoded_patterns, decoded_patterns_combined, decoded_patterns_weighed, decoded_patterns_count

In [42]:
decoded_patterns, decoded_patterns_combined, decoded_patterns_weighed, decoded_patterns_count = process_two_voices(m21_score, compared_score)

2487
2623
Query:    1 GGMFFFFGGMFMGGGMFFFFGGFFMGGEGGGFEGGGFFFGCKMMGGMFFFFGGFFMGGEGGGFEGGGFFFGCKMMGGMFFFFGGFFMJKGGEHFFFFGGCHGGCFFFGFMAHIFIGGCGGHFFFFGGCHGG-CFFFKFHFMEGGGGEGGGFEDFG-JGGFFDG-GGMGEGCIHMGGCDIMGG-HFFFGEGGCHGGCFFFKFHFMEGGGGEGGGFEDMEG-GJGGFFDGGGMGEGCIHMGGCDIMGG--HFFFGEGGBGGGGGCFFKFKFFBGGHGGCG---GBG---F-F-FFGGEEGGGGBMKFMCDGGLGGCGGBGFFF-F-GGEEGGGGEFFFF----FLFMFJFMMFFGGECGIGGGFECF-GGJG--GFFDGG-GFFGGCKCMFKMGGCGGBGFFFFGCGGGGGEDGF--FFLFMFJFMMFFGGECGIGGGFECF-GGJG--GFFDGG-GFFGGCKCMFKMGGCGGBGFFFFGCGGGGGEDGF--FFLFMFJFMIFFJGG-MFFDCMJIMFMGGGMFFDCMJIDCMJIFJIGF-M-DI-GAJF-CI--I-M---MG--GMFFDCMJIDCMHFFDIF--JIGF-MDIGAJFCIIM---MG--GMFFDCMJIDCMHF-FMKF-FMMFFKBGICFMJIBICMMKMCMMKMCKFDFHFIGEFHFIGGCMFKCMLECMLF---FFFFGGG--GGAGGGGG-GMGBAGGAGGAG-GAKGGAD--MMEKAGFJEKAGF--FGEEGGFFMEEGLFMAHMIMEMHMHMIMB----MHMCMHMBMIFGMMMCMMMMMJMDCHBKGMAGMEHEMMMMGIMMCGGBMGGMFFFGMGGMFG-FMMDMGMGMEMFJMMMBMGGIFJCEFMGCM-----------D----MGFBMM--GIF---FMBGGH-------MGMMG--GAMF-----MG-CGHBM-MLCG--GF------G---GBMMMLCGGFIAMMMLFGEFFHFDMMMG